In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas.io.json import json_normalize
import time

In [2]:
input = input('Welcome to Make your trip! \n\nWrite the country code of the country you want to visit: \nEstados Unidos: US \nArgentina: AR \nAustralia: AU \nBélgica: BE \nCanadá: CA \nFrancia: FR \nAlemania: DE \nHong Kong: HK \nJapón: JP \nMéxico: MX \nEspaña: ES \nNetherlands: NL \nRepública Checa: CZ, \nGrecia: GR \nItalia: IT')
print(input)

Welcome to Make your trip! 

Write the country code of the country you want to visit: 
Estados Unidos: US 
Argentina: AR 
Australia: AU 
Bélgica: BE 
Canadá: CA 
Francia: FR 
Alemania: DE 
Hong Kong: HK 
Japón: JP 
México: MX 
España: ES 
Netherlands: NL 
República Checa: CZ, 
Grecia: GR 
Italia: ITNL
NL


In [3]:
link = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode='+ input + '&apikey=L89khNOrYHJzWcbQdBplAQloWumEkrH9'
url = link

response = requests.get(url)
results = response.json()

flattened_data = json_normalize(results['_embedded']['events'])
data = flattened_data.rename(columns={'name':'nombre',
                                      'type':'tipo_evento',
                                      'sales.public.startDateTime':'dia_comienzo_venta',
                                      'dates.timezone':'zona_horaria',
                                      'dates.status.code':'status_venta',
                                      'url':'link'})
data2 = data[['nombre', 'tipo_evento', 'dia_comienzo_venta', 'zona_horaria', 'status_venta', 'link']]

dataset_ticketmaster = pd.DataFrame(data2)

In [4]:
place = []
country_city = []
description = []
link = []

for i in range(1, 12):
    time.sleep(1)
    url = 'https://www.viajerosporelmundo.com/explora-el-mundo/pagina/%s'
    r = requests.get(url % i).content
        
    soup = BeautifulSoup(r)
    pla = soup.select('div.texto h2')
    places = [item.text for item in pla]
    coun = soup.select('div.info-single span.localizacion')
    countries = [item.text for item in coun]
    desc = soup.select('div.texto p')
    descriptions = [item.text for item in desc]
    links = [a['href'] for a in soup.select('div.texto h2 a')]
    
    place.append(places)
    country_city.append(countries)
    description.append(descriptions)
    link.append(links)

In [5]:
placeDF = pd.Series(place)
complete_place = placeDF.explode()

country_cityDF = pd.Series(country_city)
complete_country_city = country_cityDF.explode()

descriptionDF = pd.Series(description)
complete_description = descriptionDF.explode()

linkDF = pd.Series(link)
complete_link = linkDF.explode()

In [6]:
data_viajeros = pd.concat([complete_place, complete_country_city, complete_description, complete_link], axis=1)
data_viajeros.columns = ['destino_turistico', 'pais-ciudad', 'descripcion', 'link']

In [7]:
pais_ciudad_code = {' Francia | París': 'FR', ' España | Granada': 'ES', ' Bélgica | Bruselas': 'BE',
                    ' Países Bajos | Ámsterdam': 'NL', ' Inglaterra | Londres': 'EN', ' España | Córdoba': 'ES',
                    ' Francia': 'FR', ' Francia | Altos Pirineos de Francia - Hautes-Pyrénées': 'FR',
                    ' España | Barcelona': 'ES', ' EEUU | Nueva York': 'US', ' Italia | Venecia': 'IT',
                    ' España | Madrid': 'ES', ' Italia | Roma': 'IT', ' Alemania | Berlín': 'DE',
                    ' República Checa | Praga': 'CZ', ' Grecia | Atenas': 'GR', ' Francia | Lourdes': 'FR',
                    ' Francia | Carcasona': 'FR', ' Francia | Versalles': 'FR', ' España | San Martín de la Vega':'ES'}

data_viajeros['pais-ciudad'] = data_viajeros['pais-ciudad'].replace(pais_ciudad_code)
dataset_viajeros_por_el_mundo = data_viajeros[data_viajeros['pais-ciudad'] == input]

In [8]:
dataset_viajeros_por_el_mundo

,destino_turistico,pais-ciudad,descripcion,link
0,Casas Flotantes de Ámsterdam,NL,"Ámsterdam se caracteriza por sus canales, sus ...",https://www.viajerosporelmundo.com/explora-el-...
0,Zaanse Schans: La ruta de los molinos,NL,Vas a Ámsterdam y quieres conocer uno de sus s...,https://www.viajerosporelmundo.com/explora-el-...
1,Coffe shops,NL,Ámsterdam es mundialmente conocida por sus can...,https://www.viajerosporelmundo.com/explora-el-...
3,Barrio Rojo,NL,En el corazón de Ámsterdam hay un barrio que s...,https://www.viajerosporelmundo.com/explora-el-...
4,Coffe shops,NL,Ámsterdam es mundialmente conocida por sus can...,https://www.viajerosporelmundo.com/explora-el-...
5,Casa de Ana Frank,NL,La Casa de Ana Frank es uno de los lugares más...,https://www.viajerosporelmundo.com/explora-el-...
8,Museo Van Gogh,NL,Vincent Van Gogh es uno de los artistas holand...,https://www.viajerosporelmundo.com/explora-el-...
9,Canales de Ámsterdam,NL,Los canales son a Ámsterdam lo mismo que el Pa...,https://www.viajerosporelmundo.com/explora-el-...
10,Coffe shops,NL,Ámsterdam es mundialmente conocida por sus can...,https://www.viajerosporelmundo.com/explora-el-...
10,Museo Van Gogh,NL,Vincent Van Gogh es uno de los artistas holand...,https://www.viajerosporelmundo.com/explora-el-...


In [9]:
dataset_ticketmaster

,nombre,tipo_evento,dia_comienzo_venta,zona_horaria,status_venta,link
0,Jonas Brothers - Happiness Begins Tour | Barri...,event,2019-06-06T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/jonas-brothe...
1,Jonas Brothers - Happiness Begins Tour,event,2019-06-06T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/jonas-brothe...
2,Jonas Brothers - Happiness Begins Tour | Plati...,event,2019-06-06T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/jonas-brothe...
3,Billie Eilish: WHERE DO WE GO? WORLD TOUR,event,2019-10-04T07:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/billie-eilis...
4,Celine Dion - Courage World Tour | Platinum Ti...,event,2019-09-27T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...
5,Celine Dion - Courage World Tour | The Opal VI...,event,2019-09-27T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...
6,Celine Dion - Courage World Tour | The Sapphir...,event,2019-09-27T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...
7,Celine Dion - Courage World Tour | The Emerald...,event,2019-09-27T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...
8,Celine Dion - Courage World Tour | The Ruby VI...,event,2019-09-27T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...
9,Celine Dion - Courage World Tour,event,2019-10-02T08:00:00Z,Europe/Amsterdam,onsale,https://www.ticketmaster.nl/event/celine-dion-...


In [10]:
dataset_viajeros_por_el_mundo.to_csv('destinos_viajeros_por_el_mundo.csv', index=False)
dataset_ticketmaster.to_csv('eventos_ticketmaster.csv', index=False)